In [4]:
!az extension add -n ml -y #hi this is code abe

The installed extension 'ml' is in preview.


In [ ]:
!az login

In [11]:
import subprocess

#Get MLFLow UI through the Azure ML CLI v2 and convert to string
MLFLOW_TRACKING_URI = subprocess.run(["az", "ml", "workspace", "show", "--query", "mlflow_tracking_uri", "-o", "tsv"], stdout=subprocess.PIPE, text=True)
MLFLOW_TRACKING_URI = str(MLFLOW_TRACKING_URI.stdout).strip()

ERROR: Code: ResourceGroupNotFound
Message: Resource group 'osomorog' could not be found.


In [13]:
import mlflow
from azureml.core import Workspace
mlflow.set_tracking_uri(Workspace.from_config().get_mlflow_tracking_uri())

In [14]:
import mlflow
experiment_name = 'experiment_with_mlflow'
mlflow.set_experiment(experiment_name)


In [15]:
import os
from random import random

with mlflow.start_run() as mlflow_run:
    mlflow.log_param("hello_param", "world")
    mlflow.log_metric("hello_metric", random())
    os.system(f"echo 'hello world' > helloworld.txt")
    mlflow.log_artifact("helloworld.txt")


In [16]:
from mlflow.tracking import MlflowClient
# Use MlFlow to retrieve the run that was just completed
client = MlflowClient()
run_id = mlflow_run.info.run_id
finished_mlflow_run = MlflowClient().get_run(run_id)

metrics = finished_mlflow_run.data.metrics
tags = finished_mlflow_run.data.tags
params = finished_mlflow_run.data.params

print(metrics,tags,params)


{'hello_metric': 0.2676835330691445} {'mlflow.user': 'azureuser', 'mlflow.source.name': '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.rootRunId': '932e4da8-76af-4f27-89e5-6ce83c994823'} {'hello_param': 'world'}


In [17]:
client.list_artifacts(run_id)

[<FileInfo: file_size=-1, is_dir=False, path='helloworld.txt'>]

In [18]:
client.download_artifacts(run_id, "helloworld.txt", ".")

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/osomorog3/code/Users/osomorog/MLFlow_Scenarios/helloworld.txt'

In [19]:
### Compare and query:
from mlflow.entities import ViewType

all_experiments = [exp.experiment_id for exp in MlflowClient().list_experiments()]
query = "metrics.hello_metric > 0"
runs = mlflow.search_runs(experiment_ids=all_experiments, filter_string=query, run_view_type=ViewType.ALL)

runs.head(10)


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.hello_metric,params.hello_param,tags.mlflow.rootRunId,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.user
0,9d6edb9b-f9d8-4870-bc6f-5d192c28e81a,18168c22-8581-4420-9b3b-4301464d25c3,FINISHED,,2021-10-13 00:17:53.632000+00:00,2021-10-13 00:21:00.184000+00:00,0.272088,world,9d6edb9b-f9d8-4870-bc6f-5d192c28e81a,None,None,None
1,4e10a4f8-06a1-4e4d-8423-cafb7be78371,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-16 00:59:20.789000+00:00,2021-11-16 00:59:23.846000+00:00,0.056947,world,4e10a4f8-06a1-4e4d-8423-cafb7be78371,/anaconda/envs/azureml_py36/lib/python3.6/site...,LOCAL,azureuser
2,dd459782-9e85-4544-af88-9eb492ac74ec,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-16 01:04:06.568000+00:00,2021-11-16 01:05:02.270000+00:00,0.882664,world,dd459782-9e85-4544-af88-9eb492ac74ec,None,None,None
3,26edd15b-9cce-4a30-b422-c7439aedb06a,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-16 01:22:56.810000+00:00,2021-11-16 01:22:58.738000+00:00,0.023711,world,26edd15b-9cce-4a30-b422-c7439aedb06a,/anaconda/envs/azureml_py38/lib/python3.8/site...,LOCAL,azureuser
4,ec8d02d1-dbc3-4fdb-83bd-d24e25db1df7,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-16 01:51:58.870000+00:00,2021-11-16 01:52:01.301000+00:00,0.586012,world,ec8d02d1-dbc3-4fdb-83bd-d24e25db1df7,/anaconda/envs/azureml_py38/lib/python3.8/site...,LOCAL,azureuser
5,932e4da8-76af-4f27-89e5-6ce83c994823,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-20 00:02:11.545000+00:00,2021-11-20 00:02:14.431000+00:00,0.267684,world,932e4da8-76af-4f27-89e5-6ce83c994823,/anaconda/envs/azureml_py38/lib/python3.8/site...,LOCAL,azureuser


In [21]:
# the model folder produced from the run is registered. This includes the MLmodel file, model.pkl and the conda.yaml.
model_path = "model"
model_uri = 'runs:/{}/{}'.format(run_id, model_path) 
mlflow.register_model(model_uri,"registered_model_name_2")


Successfully registered model 'registered_model_name_2'.


RestException: RESOURCE_DOES_NOT_EXIST: Response: {'Error': {'Code': 'UserError', 'Severity': None, 'Message': 'Provided model source path: model does not exist in the set of files uploaded for the run.', 'MessageFormat': None, 'MessageParameters': None, 'ReferenceCode': None, 'DetailsUri': None, 'Target': None, 'Details': [], 'InnerError': None, 'DebugInfo': None, 'AdditionalInfo': None}, 'Correlation': {'operation': '54f11e1d43111648b11ccd134bb9c9ce', 'request': 'fdd04cf524d85c43'}, 'Environment': 'westus', 'Location': 'westus', 'Time': '2021-11-16T01:04:40.7198453+00:00', 'ComponentName': 'mlflow', 'error_code': 'RESOURCE_DOES_NOT_EXIST'}